<a href="https://colab.research.google.com/github/kocurvik/edu/blob/master/PNSPV/notebooky/keras_2020/cv09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 9. Cvičenie - Rekurentné siete

Na dnešnom cvičení si precvičíme prácu s rekurentnými sieťami. Vytvoríme si mierne umelý príklad, v ktorom skonštruujeme veľké číslo zo sekvencie MNIST čísel a vrátime honotu po aplikácii modula nejakým číslom (ideálne prvočíslo nad 10). 

In [ ]:
import keras
from keras.datasets import mnist
from keras.layers import TimeDistributed, Conv2D, Dense, Flatten, MaxPool2D, ConvLSTM2D, GlobalAveragePooling2D, LSTM, GRU
from keras.models import Sequential
from keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
(x, y), (x_test, y_test) = mnist.load_data()
x = x / 255
x_test = x_test / 255

11493376/11490434 [==============================] - 2s 0us/step


## Fixná sekvencia - priebežná chyba

Navrhneme si model, ktorý na začiatku bude mať konvolučné vrstvy. Potom bude mať Globálny Pooling nasledovaný GRU (gated recurrent unit) vrstvou. Tá je prepojená aj s predchádzajúcimi vrstvami v sekvencii. Nasleduje plne prepojená vrstva so softmaxom. Známejšia je vrstva LSTM, ale GRU používame pretože sa jednoduchšie trénuje. Teoreticky by LSTM malo mať dlhodobejšiu pamäť. To však nebude v našom prípade až také dôležité.

![GRU](https://raw.githubusercontent.com/kocurvik/edu/master/PNSPV/supplementary/ntb_images/GRU_model.png)

Môžeme si vybrať ako budeme trénovať túto sieť. Na vstupe budeme mať sekvenciu obrázkov tvaru $batch \times seq \times 28 \times 28 \times 1$, kde $seq$ je dĺžka sekvencie. Na výstupe máme dve možnosti. Buď budeme sledovať len výstup na konci sekvencie a z neho rátať loss, alebo budeme trénovať sieť na všetky vstupy. Najprv otestujeme druhý prístup:

![GRU Multiloss](https://raw.githubusercontent.com/kocurvik/edu/master/PNSPV/supplementary/ntb_images/GRU_multi_loss.png)

Model je definovaný nižšie. Wrapper `TimeDistributed` nám umožňuje zavolať nejaký druh vrsvty samostatne na po jednotlivých častiach sekvencie. Pri rekurentnej vrstve tento wrapper nepotrebujeme!

In [ ]:
def build_model(modulo, seq_size):
  model = Sequential()
  model.add(TimeDistributed(Conv2D(16, (7,7), activation='relu'), input_shape=(seq_size, 28, 28, 1)))
  model.add(TimeDistributed(Conv2D(32, (7,7), activation='relu')))
  model.add(TimeDistributed(Conv2D(64, (7,7), activation='relu')))
  model.add(TimeDistributed(Conv2D(64, (7,7), activation='relu')))
  model.add(TimeDistributed(GlobalAveragePooling2D()))
  model.add(GRU(64, activation='relu', recurrent_activation='hard_sigmoid', return_sequences=True))
  model.add(TimeDistributed(Dense(modulo, activation='softmax')))

  return model

Teraz bude nutné naprogramovať generátor ktorý nám dá vstupné dáta a výstupné dáta na loss. Metódu getitem doimplementujte. Na konci chceme mať klasifikáciu do `self.modulo` tried tak, že zoberieme čísla ktoré sme dávali do siete akoby sme ich postupne (zľava) zapisovali a vyrátame ich modulo našim zvoleným číslom. Defaultne 13kou.

In [ ]:
class SeqDataGenerator(keras.utils.Sequence):
  def __init__(self, x, y, modulo=13, batch_size=32, seq_size=20, steps=1000):
    self.x = x
    self.y = y
    self.modulo = modulo
    self.batch_size = batch_size
    self.seq_size = seq_size
    self.steps = steps

  def __len__(self):
    return self.steps

  def __getitem__(self, index):
    X = np.empty([self.batch_size, self.seq_size, 28, 28, 1])
    y = np.zeros([self.batch_size, self.seq_size, self.modulo])

    ## doimplemtujte

    return X, y

Tento kód by mal potom natrénovať našu sieť na dátach. Môžete sa pohrať s nastavením dĺžky sekvencie.

In [ ]:
seq_size = 10
modulo = 13

train_gen = SeqDataGenerator(x[10000:], y[10000:], modulo=modulo, batch_size=32, seq_size=seq_size)
val_gen = SeqDataGenerator(x[:10000], y[:10000], modulo=modulo, batch_size=32, seq_size=seq_size, steps=100)
model = build_model(modulo, seq_size)

opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

model.fit_generator(train_gen, validation_data=val_gen, epochs=5)

## Fixná sekvencia - Chyba len na konci

Teraz vyskúšame trénovanie tak, že chybu budeme počítať len z konca celej sekvencie.

![alt text](https://raw.githubusercontent.com/kocurvik/edu/master/PNSPV/supplementary/ntb_images/GRU_single_loss.png)

Doimplemenujte relevantné metódy. Zistite čo robí argument `return_sequential` pri GRU vrstve a sieť upravte tak aby na výstupe bol len jeden vektor. Otestujte či trénovanie funguje a otestujte to pre rôzne dĺžky sekvencie.

In [ ]:
def build_single_model(modulo, seq_size):
  # doimplementujte
  ...

In [ ]:
class SingleSeqDataGenerator(keras.utils.Sequence):
  def __init__(self, x, y, modulo=11, batch_size=32, seq_size=20, steps=1000):
    self.x = x
    self.y = y
    self.modulo = modulo
    self.batch_size = batch_size
    self.seq_size = seq_size
    self.steps = steps

  def __len__(self):
    return self.steps

  def __getitem__(self, index):
    X = np.empty([self.batch_size, self.seq_size, 28, 28, 1])
    y = np.zeros([self.batch_size, self.modulo])

    for b in range(self.batch_size):
      # doimplementujte

    return X, y

In [ ]:
seq_size = 6
modulo = 13

train_gen = SingleSeqDataGenerator(x[10000:], y[10000:], modulo=modulo, batch_size=32, seq_size=seq_size)
val_gen = SingleSeqDataGenerator(x[:10000], y[:10000], modulo=modulo, batch_size=32, seq_size=seq_size, steps=100)
single_model = build_single_model(modulo, seq_size)

opt = Adam(lr=0.001)
single_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

single_model.fit_generator(train_gen, validation_data=val_gen, epochs=5)

## Rôzne dĺžky sekvencie

Teraz vyskúšame trénovať model vždy na inú dĺžku sekvencie. Minibatch necháme rovnaký, ale ak by sme mali problém s pamäťou tak ho môžeme redukvať. V definícii modelu, len v input shape dáme ako prvú dimenziu None

In [ ]:
def build_var_model(modulo):
  model = Sequential()
  model.add(TimeDistributed(Conv2D(16, (7,7), activation='relu'), input_shape=(None, 28, 28, 1)))
  model.add(TimeDistributed(Conv2D(32, (7,7), activation='relu')))
  model.add(TimeDistributed(Conv2D(64, (7,7), activation='relu')))
  model.add(TimeDistributed(Conv2D(64, (7,7), activation='relu')))
  model.add(TimeDistributed(GlobalAveragePooling2D()))
  model.add(GRU(64, activation='relu', recurrent_activation='hard_sigmoid', return_sequences=True))
  model.add(TimeDistributed(Dense(modulo, activation='softmax')))

  return model

Náš generátor teraz upravíme aby sme vždy vybrali náhodnú dĺžku sekvencie.

*Pozn.:* Sekvencia musí byť dĺžky aspoň 2 aby to tensorflow zvládol.

In [ ]:
class VarSeqDataGenerator(keras.utils.Sequence):
  def __init__(self, x, y, modulo=11, batch_size=32, max_seq_size=20, steps=1000):
    self.x = x
    self.y = y
    self.modulo = modulo
    self.batch_size = batch_size
    self.max_seq_size = max_seq_size
    self.steps = steps

  def __len__(self):
    return self.steps

  def __getitem__(self, index):
    seq_size = np.random.randint(2, self.max_seq_size)   
    X = np.empty([self.batch_size, seq_size, 28, 28, 1]) 
    y = np.zeros([self.batch_size, seq_size, self.modulo])

    # doimplementujte

    return X, y

Pri takomto prístupe ale môžeme do siete napr. pre validačné dáta poskytnúť generátor dĺžky 10.

In [ ]:
max_seq_size = 20
modulo = 13

train_gen = VarSeqDataGenerator(x[10000:], y[10000:], modulo=modulo, batch_size=32, max_seq_size=max_seq_size)
val_gen = SeqDataGenerator(x[:10000], y[:10000], modulo=modulo, batch_size=32, seq_size=10, steps=100)
var_model = build_var_model(modulo)

opt = Adam(lr=0.001)
var_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

print(var_model.summary())

var_model.fit_generator(train_gen, validation_data=val_gen, epochs=5)

## Inferencia celou sekvenciou

Do posledného var modelu môžeme hodiť sekvenciu ľubovolnej dĺžky. Môžeme si tak model overiť.

In [ ]:
seq_size = 10
X = np.empty([1, seq_size, 28, 28, 1])

sum = 0

for s in range(seq_size):
  idx = np.random.randint(10000)
  X[0, s, :, :, 0] = x_test[idx]
  sum = sum*10 + y_test[idx]
  plt.imshow(x_test[idx], cmap='gray')
  plt.show()
  print("Current: {}, sum: {}, mod: {}".format(y_test[idx], sum, sum % 13))

pred = var_model.predict(X)
pred_last = np.argmax(pred[0], axis=-1)
print(pred_last)


## Inferencia po častiach

Chceli by sme do modelu postupne vkladať obrázky a priebežne vidieť ako sa nám vyvíja modulo súčtu. Na to však musíme model upraviť na tzv. stateful. To znamená, že GRU vrsva si pamätá výstup z predchádzajúceho príkladu. Ak máme takýto stav tak pri ďalšom použití musíme model resetovať.

Na tento účel môžeme použiť šikovnú funkciu:

In [ ]:
import json
from keras.models import model_from_json

def convert_to_inference_model(original_model):
    original_model_json = original_model.to_json()
    inference_model_dict = json.loads(original_model_json)
    print(inference_model_dict)

    layers = inference_model_dict['config']['layers']
    for layer in layers:
        if 'stateful' in layer['config']:
            layer['config']['stateful'] = True

        if 'batch_input_shape' in layer['config']:
            layer['config']['batch_input_shape'][0] = 1
            layer['config']['batch_input_shape'][1] = 1

    inference_model = model_from_json(json.dumps(inference_model_dict))
    inference_model.set_weights(original_model.get_weights())

    return inference_model

In [ ]:
inference_model = convert_to_inference_model(var_model)

sum = 0

for s in range(20):
  idx = np.random.randint(10000)
  img = x_test[idx]
  # doimplementujte

Ak chceme model použíť odznova, tak musíme stavy v GRU resetovať.

In [ ]:
sum = 0

inference_model.reset_states()

for s in range(20):
  idx = np.random.randint(10000)
  img = x_test[idx]
  # doimplementujte